In [44]:
import numpy as np
import pandas as pd
import bz2
import re

In [45]:
base_path = '../data'
raw_path = f'{base_path}/raw'
vectorize_path = f'{base_path}/vectorize'

dataset = 'mix'
batch_size_file = 1000

In [46]:
def save_csv(label, review_text, subset, batch_idx):
  df = pd.DataFrame({
    'review_text': review_text,
    'label': label
  })
  
  df['review_text'].to_csv(f'{raw_path}/{dataset}/{subset}/X_{batch_idx}.csv', index=False)
  df['label'].to_csv(f'{raw_path}/{dataset}/{subset}/y_{batch_idx}.csv', index=False)

In [47]:
def split_data(file, subset, batch_size):
  i = 0
  batch_idx = 0
  labels=[]
  texts=[]
  for line in bz2.BZ2File(file):
    i = i + 1
    x=line.decode()
    labels.append(int(x[9])-1)
    text = x[11:].strip()
    text = re.compile(r"[^a-z0-1\s]").sub(r" ",re.compile(r"['\W']").sub(r" ",text.lower()))
    texts.append(text)
    
    if i % batch_size == 0 :
      batch_idx = i//batch_size - 1
      save_csv(labels, texts, subset, batch_idx)
      labels = []
      texts = []
      
  batch_idx = batch_idx + 1
  save_csv(labels, texts, subset, batch_idx)
  num_batch = batch_idx + 1

In [48]:
split_data("../data/raw/mix/training_set/train.ft.txt.bz2", 'training_set', batch_size_file)
split_data("../data/raw/mix/testing_set/test.ft.txt.bz2", 'testing_set', batch_size_file)